In [1]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import gensim, logging, os, re, string, tensorflow
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
from gensim.utils import simple_preprocess

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
file = open("movies.txt", errors = 'ignore')
x = [word for word in file if (word.startswith('review/text') or word.startswith('product/productId') )]

In [12]:
tags = x[::2]
movies = x[1::2]

In [46]:
data = pd.DataFrame(
    {'product': tags,
     'movies': movies
    })

In [59]:
a = data.groupby("product").filter(lambda x: len(x) > 200)
a.head()

,product,movies
71,product/productId: B000063W1R\n,review/text: This is one of my all-time favori...
72,product/productId: B000063W1R\n,review/text: LOVE this movie - loved it when i...
73,product/productId: B000063W1R\n,review/text: This movie has everything I like:...
74,product/productId: B000063W1R\n,review/text: Great Blu-ray DVD for the price! ...
75,product/productId: B000063W1R\n,"review/text: I love this movie! Sure, the acti..."


In [63]:
a['product'].value_counts()

product/productId: B002QZ1RS6\n    957
product/productId: B007FQDPL8\n    956
product/productId: B001KZG99A\n    925
product/productId: B001GAPC1K\n    925
product/productId: B00005JPS8\n    925
product/productId: B001I1NGHY\n    925
product/productId: B001C08RHA\n    925
product/productId: B001FD5KJM\n    925
product/productId: B004JMSIRC\n    924
product/productId: B001FZ9AAU\n    924
product/productId: B001HUHBE0\n    924
product/productId: B000H7I6CU\n    904
product/productId: B002LW9ITY\n    902
product/productId: B000ENV5WK\n    902
product/productId: 7883685635\n    902
product/productId: B004I654WQ\n    902
product/productId: B004OBQDJ8\n    902
product/productId: B000V1Y43W\n    902
product/productId: B000H5U832\n    902
product/productId: B0049J3QF4\n    902
product/productId: B00005JNS0\n    902
product/productId: B005ZMUQCK\n    902
product/productId: B001DWNUE2\n    886
product/productId: B008UBYTRG\n    886
product/productId: B000S6LS66\n    886
product/productId: B000W4

In [65]:
atags = a["product"].values
amovies = a["movies"].values

In [68]:
amovies[0]

"review/text: This is one of my all-time favorite movies for escaping from worldly work.  Jim Caviezel and Guy Pierce have great chemistry and the trials of Edmond Dantes are laid out (although too brief for me) in historically correct detail.<br />Even though this movie loosely follows Dumas' book, the story has such a karmic ending that it always lifts my sprits.\n"

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[atags[i]]) for i, _d in enumerate(amovies[:10000])]

In [75]:
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\doc2vec.py:579: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [76]:
print("Word2Vec vocabulary length: ", len(model.wv.vocab))

Word2Vec vocabulary length:  45628


In [77]:
tagged_data[0]

TaggedDocument(words=['review/text', ':', 'this', 'is', 'one', 'of', 'my', 'all-time', 'favorite', 'movies', 'for', 'escaping', 'from', 'worldly', 'work', '.', 'jim', 'caviezel', 'and', 'guy', 'pierce', 'have', 'great', 'chemistry', 'and', 'the', 'trials', 'of', 'edmond', 'dantes', 'are', 'laid', 'out', '(', 'although', 'too', 'brief', 'for', 'me', ')', 'in', 'historically', 'correct', 'detail.', '<', 'br', '/', '>', 'even', 'though', 'this', 'movie', 'loosely', 'follows', 'dumas', "'", 'book', ',', 'the', 'story', 'has', 'such', 'a', 'karmic', 'ending', 'that', 'it', 'always', 'lifts', 'my', 'sprits', '.'], tags=['product/productId: B000063W1R\n'])

In [81]:
similar_doc = model.docvecs.most_similar('product/productId: B000063W1R\n')
print(similar_doc)

[('product/productId: B0001G6PZC\n', 0.37979960441589355), ('product/productId: B000FVQLQQ\n', 0.33286386728286743), ('product/productId: 5556167281\n', 0.26530563831329346), ('product/productId: B00151QYU8\n', 0.25443241000175476), ('product/productId: 6304675771\n', 0.25094324350357056), ('product/productId: B0000DK4QK\n', 0.21609629690647125), ('product/productId: B0002B55DE\n', 0.19313490390777588), ('product/productId: B0000DK4QI\n', 0.1890326738357544), ('product/productId: B002YJMMBA\n', 0.1720971316099167), ('product/productId: B001NFNFMQ\n', 0.09964695572853088)]
